### Enter full names of group members:

##### Name A: Dominik Ucher
##### Name B:

In [1]:
import math
import numpy as np
from sympy import prime
from pathlib import Path  # for paths of files
import csv
import copy
import random
# from sklearn.metrics.pairwise import cosine_similarity

# ANSI escape codes for colors
class colors:
    red = '\033[91m'
    green = '\033[92m'
    blue = '\033[94m'
    end = '\033[0m'  

### 1. DGIM

#### 1.1. DGIM algorithm

In [2]:
# Default DGIM parameters

stream_path = 'data/my_stream.txt'

# The window size
N = 500 

In [3]:
def checkBucket(bucketList, end_time_stamp):
    for i in range (len(bucketList) - 1):
        if len(bucketList[i]) > 2:
            oldtime = bucketList[i].pop(0)
            secondoldtime = bucketList[i].pop(0)
            bucketList[i+1].append(secondoldtime)

def checkTime(bucketList, end_time_stamp):
    for bucket in bucketList:
        for time in bucket:
            if (end_time_stamp - time) > N:
                bucket.remove(time)


def dgim_algorithm(stream_path, N):
    
    # Create the buckets and initialize the timestamp
    k = int(N).bit_length()
    bucket_list = [[] for _ in range(k)]
    end_time_stamp = 0
    
    # Loop through the entire data stream, one bit at a time
    with open(stream_path) as f:
        while True:
            bit = f.read(1)
            
            # Clause to break while loop at the end of the stream
            if not bit:
                break

            # To-do! update timestamp
            end_time_stamp += 1
            
            # To-do! implement the dgim algorithm here
            if bit == '1':
                bucket_list[0].append(end_time_stamp)
                checkBucket(bucket_list, end_time_stamp)
                checkTime(bucket_list, end_time_stamp)

        # for i, bucket in enumerate(bucket_list):
        #     bucket_list[i] = [time % N for time in bucket]

    return bucket_list, end_time_stamp 

In [4]:
bucket = dgim_algorithm(stream_path, N)

In [5]:
print(f"The updated list of timestamps buckets from DGIM algorithm: \n {bucket[0]}")
print(f"The end timestamp: {bucket[1]}")  

The updated list of timestamps buckets from DGIM algorithm: 
 [[1010099], [1010091, 1010096], [1010083, 1010089], [1010063, 1010075], [1010044], [1010006], [1009821, 1009946], [1009688], []]
The end timestamp: 1010102


#### 1.2. Query the Bucket 

In [6]:
def actual_count(stream_path, k):
    stream_list = []
    with open(stream_path, 'r') as file:
        for line in file:
            stream_list.extend(list(map(int, line.strip())))

    # Convert the list into a numpy array
    stream_array = np.array(stream_list)
    
    return int(np.sum(stream_array[-k:]))

In [10]:
def dgim_query(bucket, N, k):  

    one_count = 0
    end_time_stamps = bucket[0]
    current_time = bucket[1] - 2
    min_time = current_time - k
    valid_times = []

    for i in range(len(end_time_stamps)):
        for bucket in end_time_stamps[i]:
            if min_time <= bucket:
                valid_times.append(2**i)

    valid_times[-1] = valid_times[-1] / 2

    one_count = sum(valid_times)
    
    
    return math.ceil(one_count)

In [11]:
# List of queries
K = [10, 50, 100, 300, 500] 

In [12]:
print("---------------------------------------------------------------")
for k in K:
    dgim_count = dgim_query(bucket, 500, k)
    true_count = actual_count(stream_path, k)
    
    print(f"The total 1s in the last {k} bits by DGIM: {dgim_count}")
    print(f"The true count of 1s in the last {k} bits: {true_count}")
    print(f"The DGIM error for predicted 1s in the last {k} bits: \
    {round(abs(100*(dgim_count-true_count))/true_count,2)} %")
    print("---------------------------------------------------------------")

---------------------------------------------------------------
The total 1s in the last 10 bits by DGIM: 4
The true count of 1s in the last 10 bits: 5
The DGIM error for predicted 1s in the last 10 bits:     20.0 %
---------------------------------------------------------------
The total 1s in the last 50 bits by DGIM: 25
The true count of 1s in the last 50 bits: 26
The DGIM error for predicted 1s in the last 50 bits:     3.85 %
---------------------------------------------------------------
The total 1s in the last 100 bits by DGIM: 61
The true count of 1s in the last 100 bits: 51
The DGIM error for predicted 1s in the last 100 bits:     19.61 %
---------------------------------------------------------------
The total 1s in the last 300 bits by DGIM: 173
The true count of 1s in the last 300 bits: 150
The DGIM error for predicted 1s in the last 300 bits:     15.33 %
---------------------------------------------------------------
The total 1s in the last 500 bits by DGIM: 269
The true 

### 2. Bloom filters

In [13]:
# Username data for the creation of bloom filters - B
data_file = (Path("data/bloom_username").with_suffix('.csv'))

# Test data to check the functionality and false positive rate
test1_file = (Path("data/test1_username").with_suffix('.csv'))
test2_file = (Path("data/test2_username").with_suffix('.csv'))

# Default bloom filter parameters
bloom_size = 1500000 # parameter N
h = 3 # number of hash functions

In [14]:
# create an array of bloom filter with zeros
B = np.zeros(bloom_size)

In [15]:
B

array([0., 0., 0., ..., 0., 0., 0.])

#### 2.1. Create Bloom filter

In [16]:
import sympy

def generate_hash(h, N):
    hash_list = []
    
    # To-do! generate a list of hash functions
    for _ in range(h):
        prime = sympy.randprime(1, 10000)

        def hash_func(x, prime=prime, N=N):
            hash_value = 0
            for i, char in enumerate(x):
                hash_value = (hash_value + ord(char) * (prime ** (i + 1))) % N
            return hash_value
        hash_list.append(hash_func)
        
    return hash_list

In [17]:
hashes = generate_hash(h, bloom_size)

In [18]:
def create_bloom_filter(B, hashes, data):
    with data.open(encoding='utf-8') as f:
        for name in f:
            
            # To-do! update the hash index of the bloom filter with 1s
            name = name.strip()
            for hash_func in hashes:
                index = hash_func(name)
                B[index] = 1
            

            
    return B

In [19]:
bloom_array = create_bloom_filter(B, hashes, data_file)

In [20]:
bloom_array


array([1., 1., 0., ..., 1., 1., 1.])

#### 2.2. Verify usernames

In [21]:
def single_verify_username(bloom_array, hashes, new_user):
    
    # To-do! verify username and return a code of 0 or 1 (1 - username taken and 0 - username available)
    for hash_func in hashes:
        index = hash_func(new_user)
        if (bloom_array[index] == 0):
            return 0
    
        
    return 1
    

In [22]:
# Feel free to test different usernames here

new_username = "KazeemTDT4305"

# new_username = "ShambaTDT4305"

In [23]:
user_code = single_verify_username(bloom_array, hashes, new_username)

In [24]:
if user_code == 1:
    print(colors.red + f"Username {new_username} has been taken. Try again!" + colors.end)
elif user_code == 0:
    print(colors.green + f"Username {new_username} is available. Congrats!" + colors.end)
else:
    print(colors.blue + f"Wrong pass code. Please reverify!" + colors.end)  

Username KazeemTDT4305 is available. Congrats!


In [25]:
def group_verify_username(bloom_array, hashes, data):
    # Initialize counts
    total_name = 0
    taken_name = 0
    
    with data.open(encoding = 'utf-8') as f:
        for name in f:
            # To-do! similar to the single verify, but returns a percentage of usernames taken...
            # ...(In other words seen already by the bloom filter during its creation)
            name = name.strip()
            total_name += 1
            is_taken = True

            for hash_func in hashes:
                index = hash_func(name)
                if bloom_array[index] == 0:
                    is_taken = False
                    break 
            
            if is_taken:
                taken_name += 1
                
            
    return round(taken_name/total_name*100,2)   

In [26]:
print("----------------------------------------------------------")
user_total = group_verify_username(bloom_array, hashes, test1_file)
print(f"Percentage of username seen before from test 1: {user_total}%")
print("----------------------------------------------------------")
user_total = group_verify_username(bloom_array, hashes, test2_file)
print(f"Percentage of username seen before from test 2: {user_total}%")
print("----------------------------------------------------------")

----------------------------------------------------------
Percentage of username seen before from test 1: 100.0%
----------------------------------------------------------
Percentage of username seen before from test 2: 23.74%
----------------------------------------------------------


### 3. Flajolet-Martin

In [27]:
def flajolet_martin(input_stream):
    R = 0  # Initialize maximum rightmost zero bit position to 0

    # To-do! Define hash function h(x) = 6x + 1 mod 5
    def hash_func(x):
        return (6*x + 1) % 5
    

    # To-do! Iterate over the input stream and update maximum rightmost zero bit position
    
    def trailing_zeros(x):
        if x == 0:
            return 1
        count = 0
        while x & 1 == 0:
            count += 1
            x >>= 1
        return count
    
    for element in input_stream:
        hash_val = hash_func(element)
        R = max(R, trailing_zeros(hash_val))
    

    # Estimate the number of distinct elements
    distinct_estimate = 2 ** R

    return distinct_estimate

In [28]:
# Input stream
input_stream1 = [1, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1]
input_stream2 = [1, 3, 2, 1, 2, 3, 4, 3, 1, 2, 3, 1]

# Run the Flajolet-Martin algorithm
distinct_estimate1 = flajolet_martin(input_stream1)
distinct_estimate2 = flajolet_martin(input_stream2)

# Print the estimated number of distinct elements
print("-----------------------------------------------------")
print(f"Distinct elements (estimated) in input stream 1:", distinct_estimate1)
print("-----------------------------------------------------")
print(f"Distinct elements (estimated) in input stream 2:", distinct_estimate2)
print("-----------------------------------------------------")

-----------------------------------------------------
Distinct elements (estimated) in input stream 1: 2
-----------------------------------------------------
Distinct elements (estimated) in input stream 2: 4
-----------------------------------------------------


### 4. Adword 

#### 4.1. Greedy Algorithm

In [29]:
# User queries
queries = ["big data", "big data", "big data","bloom filters", "bloom filters", "bloom filters",
           "flajolet martin", "flajolet martin", "flajolet martin", "dgim algorithm", "dgim algorithm", "dgim algorithm"]

In [30]:
# Company A B C and D keywords and budget $$$
global_companies = {
        'A': ["big data", "bloom filters", 3],
        'B': ["flajolet martin", 3],
        'C': ["flajolet martin", "dgim algorithm", 3],
        'D': ["big data", 3],
    }

In [31]:
def greedy_algorithm(local_companies, queries):
    # Initial revenue
    revenue = 0
    
    # To-do! update revenue using greedy algorithm
    for word in queries:
        valid_companies = [company for company in local_companies if word in local_companies[company][:-1] and local_companies[company][-1] > 0]
        if valid_companies:
            random_company = random.choice(valid_companies)
            local_companies[random_company][-1] -= 1
            revenue += 1

    return revenue

In [32]:
total_revenue = 0
total_trials = 10
print("Starting trials using Greedy Algorithm...")
print("------------------------------------------------")
for i in range(total_trials):
    local_companies = copy.deepcopy(global_companies)
    revenue = greedy_algorithm(local_companies, queries)
    total_revenue = total_revenue + revenue
    print(f"Trial {i+1} - Revenue generated: {revenue}")
print("------------------------------------------------")   
print("Average revenue generated for all trials: ",total_revenue/total_trials)

Starting trials using Greedy Algorithm...
------------------------------------------------
Trial 1 - Revenue generated: 9
Trial 2 - Revenue generated: 12
Trial 3 - Revenue generated: 8
Trial 4 - Revenue generated: 10
Trial 5 - Revenue generated: 8
Trial 6 - Revenue generated: 7
Trial 7 - Revenue generated: 10
Trial 8 - Revenue generated: 8
Trial 9 - Revenue generated: 7
Trial 10 - Revenue generated: 9
------------------------------------------------
Average revenue generated for all trials:  8.8


#### 4.2. Balance Algorithm

In [33]:
def balance_algorithm(local_companies, queries):
    # Initial revenue
    revenue = 0
    
    # To-do! update revenue using balance algorithm
    for word in queries:
        valid_companies = [company for company in local_companies if word in local_companies[company][:-1] and local_companies[company][-1] > 0]
        if valid_companies:
            max_budget = 0
            max_company = []
            for company in valid_companies:
                if local_companies[company][-1] > max_budget:
                    max_company = [company]
                    max_budget = local_companies[company][-1]
                elif local_companies[company][-1] == max_budget:
                    max_company.append(company)
            if max_company:
                selected_max_company = random.choice(max_company)
                revenue += 1
                local_companies[selected_max_company][-1] -= 1

    # for word in queries:
    #     valid_companies = [company for company in local_companies if word in local_companies[company][:-1] and local_companies[company][-1] > 0]
    #     if valid_companies:
    #         max_budget = 0
    #         max_company = ""
    #         for company in valid_companies:
    #             if local_companies[company][-1] > max_budget:
    #                 max_company = company
    #                 max_budget = local_companies[company][-1]
    #         if max_company:
    #             revenue += 1
    #             local_companies[max_company][-1] -= 1

    
    return revenue

In [34]:
total_revenue = 0
total_trials = 10
print("Starting trials using Balance Algorithm...")
print("-------------------------------------------")
for i in range(total_trials):
    local_companies = copy.deepcopy(global_companies)
    revenue = balance_algorithm(local_companies, queries)
    total_revenue = total_revenue + revenue
    print(f"Trial {i+1} - Revenue generated: {revenue}")
print("-------------------------------------------")   
print("Average revenue generated for all trials: ",total_revenue/total_trials)

Starting trials using Balance Algorithm...
-------------------------------------------
Trial 1 - Revenue generated: 10
Trial 2 - Revenue generated: 9
Trial 3 - Revenue generated: 9
Trial 4 - Revenue generated: 8
Trial 5 - Revenue generated: 8
Trial 6 - Revenue generated: 9
Trial 7 - Revenue generated: 9
Trial 8 - Revenue generated: 10
Trial 9 - Revenue generated: 9
Trial 10 - Revenue generated: 9
-------------------------------------------
Average revenue generated for all trials:  9.0


### 5. Recommender System

In [35]:
# Ratings matrix (each row corresponds to a movie, and each column corresponds to a user)
ratings_matrix = np.array([
    [1, 0, 3, 0, 0, 5, 0, 0, 5, 0, 4, 0],
    [0, 0, 5, 4, 0, 0, 4, 0, 0, 2, 1, 3],
    [2, 4, 0, 1, 2, 0, 3, 0, 4, 3, 5, 0],
    [0, 2, 4, 0, 5, 0, 0, 4, 0, 0, 2, 0],
    [0, 0, 4, 3, 4, 2, 0, 0, 0, 0, 2, 5],
    [1, 0, 3, 0, 3, 0, 0, 2, 0, 0, 4, 0]
])

#### 5.1. User-User Collaborative Filtering

In [36]:
from numpy.linalg import norm
import heapq

def user_cf(rate_m, tup_mu, neigh):
    
    # To-do! implement a user-user CF using cosine similarity as distance measure
    movie_num = tup_mu[0] - 1
    user_num = tup_mu[1] - 1
    sim_rating = {}

    user_row = rate_m[:,user_num]
    for i in range(len(rate_m[0])):
        A = user_row
        B = rate_m[:,i]
        cosine = np.dot(A,B)/(norm(A)*norm(B))
        sim_rating[i] = cosine

    N_highest = heapq.nlargest(neigh+1, sim_rating.items(), key=lambda item: item[1])
    N_highest = N_highest[1:]

    top = sum(rate_m[movie_num][index] * value for index,value in N_highest)
    bottom = sum(value for _, value in N_highest)
    
    prediction = top / bottom
    
    return round(prediction,2)

In [37]:
# List of tuple of movie rating by users to be predicted e.g (1, 5) refers to the rating of movie 1 by user 5
list_mu_query = [(1, 5), (3, 3)]

# Neighbor selection (|N|)
neigh = 2

In [40]:
print("-----------------------------------------------------------------")   
for mu_query in list_mu_query:
    predicted_rating = user_cf(ratings_matrix, mu_query, neigh)
    print(f"The predicted rating of movie {mu_query[0]} by user {mu_query[1]}: {predicted_rating} (User-User CF)")
    print("-----------------------------------------------------------------")   

-----------------------------------------------------------------
The predicted rating of movie 1 by user 5: 1.42 (User-User CF)
-----------------------------------------------------------------
The predicted rating of movie 3 by user 3: 1.49 (User-User CF)
-----------------------------------------------------------------


#### 5.2. Item-Item Collaborative Filtering

In [41]:
def item_cf(rate_m, tup_mu, neigh):
    
    # To-do! implement a item-item CF using cosine similarity as distance measure
    movie_num = tup_mu[0] - 1
    user_num = tup_mu[1] - 1
    sim_rating = {}

    movie_row = rate_m[movie_num]
    for i in range(len(rate_m)):
        A = movie_row
        B = rate_m[i]
        cosine = np.dot(A,B)/(norm(A)*norm(B))
        sim_rating[i] = cosine
    
    N_highest = heapq.nlargest(neigh+1, sim_rating.items(), key=lambda item: item[1])
    N_highest = N_highest[1:]

    top = sum(rate_m[index][user_num] * value for index,value in N_highest)
    bottom = sum(value for _, value in N_highest)

    prediction = top / bottom
    
    return round(prediction,2)

In [42]:
print("-----------------------------------------------------------------")   
for mu_query in list_mu_query:
    predicted_rating = item_cf(ratings_matrix, mu_query, neigh)
    print(f"The predicted rating of movie {mu_query[0]} by user {mu_query[1]}: {predicted_rating} (Item-Item CF)")
    print("-----------------------------------------------------------------")   

-----------------------------------------------------------------
The predicted rating of movie 1 by user 5: 2.48 (Item-Item CF)
-----------------------------------------------------------------
The predicted rating of movie 3 by user 3: 3.0 (Item-Item CF)
-----------------------------------------------------------------


### Provide concise answers to all 5 cases in the Project 3 description below

#### Case 1

#### Question
DGIM stores $O(\log^2 N)$ bits rather than the entire window N. Knowing that the number of
buckets used in the DGIM algorithm is log N, why then is the space complexity $O(\log^2 N)$ rather than
$O(\log N)$?

#### Answer
The space complexity of the DGIM algorithm is $O(\log^2 N)$ rather than $O(\log N)$. This is because we store $O(\log^2 N)$ bits per window where N = Window Size. Given that the record stamps are in modula N (The window size), we can represent any relevant timestamps with $O(\log^2 N)$ bits. Therefore we must have the space complexity of $O(\log^2 N)$ in the DGIM algorithm rather than $O(\log N)$, so that it will work properly and be able to save all relevant timestmaps and not miss any data


#### Case 2

#### Question:
**1.** Your favorite social media platform uses Bloom filters to assign new usernames. When
you try to obtain a new username, ’Kazeem,’ it says, "username is taken." Is there a possibility that the username is available? If yes, how do you present your case to the site admin?

**2.** After a bit of ingenuity on your part, you finally came up with ’KazeemTDT4305", and fortunately, it says this is available. However, a friend sitting next to you prompted that they know a friend of a friend that has this exact username on the same platform. Is this possible? Knowing that this friend has never taken a CS course on Big Data and has probably always doubted your brilliance. Clearly, this is the moment you have been waiting for to brandish your TDT4305 Big Data prowess. How can you effectively convince your friend that he is mistaken?

**3. While you are at it, show a bit of altruism by solving this case for a Google user Google Support Center:**

I am trying to create an account and when I use my daughter's first.last@gmail.com, google says that account is taken. I found that hard to believe so I emailed the account and sure enough, it bounced back saying no such user.

Any idea why this is and how I can contact google to make an exception?

I was able to do this for my son, but not for my daughter.


#### Answer
**1.** Yes, when using Bloom Filters there is a possibility that the username is still available even though it says it is taken. That it because of the hashing method of the bloom filters they can have false positives but no false negatives. This is so that there will be no duplicate username by mistake. It could also be because of the hashing method and there is a username that is similar, then becuase of the no false negatives method in the bloom filters, it will say the username is taken

**2.** As said above, bloom filters work by first hashing each letter/number value in you username, sum up the hashed values and indexes them. This way, Bloom filters can never have any false positives (That it says that the username is available when it is not available). Because when it checks if the username is available or not, it hashes the username and checks the index number. If it is 0 at the index number then it is available, if it is 1 then it is not available. This way it checks much faster, although it could be that you username is available when it says it is not available, because we only check the hash values index number and not the username.

**3.** It says that your username is taken because of the way Google checks already made usernames. That means that there is another username (that is different) but has the same "Google Value" as yours. So the fastest way is to add a number at the end of the name. Then it will get another "Google Value" that will most likely be available. If not then just get in touch with Google, so they can manually check if the username is taken or not.

#### Case 3

#### Question
How do we increase precision while using the Flajolet-Martin algorithm?

#### Answer
You can increase the precision by grouping hash functions, take median of value obtained in each group and then the average value obtained across groups.
You can also repeat many times and compute in parallel for multiple hash functions. This will lead to a more precise Flajolet-Martin algorithm.


#### Case 4

#### Question
What is the minimum and maximum possible revenue, and the competitive ratio of both the
Greedy and Balance algorithm for the input data provided in the Notebook?

#### Answer
To answer this question, we write a code that runs both algorithms 100 times and returns the minimum/maximum value and competitive ratio as given with the Competitive Ratio Equitation:
$$
\text{Competitive ratio} = \min_{\text{all possible inputs}} \left( \frac{|M_{\text{greedy}}|}{|M_{\text{opt}}|} \right)
$$

#### Greedy Algorithm Results:

*   **Maximum Value**: 11
*   **Minimum Value**: 6
*   **Competitive Ratio**: $$ \frac{6}{12} = \frac{1}{2} = 0.5 $$

#### Balance Algorithm Results:

*   **Maximum Value**: 10
*   **Minimum Value**: 8
*   **Competitive Ratio**: $$ \frac{8}{12} = \frac{2}{3} \approx 0.667 $$

P.S. Chose to run it 100 times

#### Case 5

#### Question
Observe the rating matrix, based on intuition, which gave a better prediction for movie 1 and user 5 rating between User-User CF and Item-Item CF.

#### Answer

**User-User Collaborative Filtering (CF):**

- Predicted rating of movie 1 by user 5: **1.42**

**Item-Item Collaborative Filtering (CF):**

- Predicted rating of movie 1 by user 5: **2.48**

**Input Matrix:**

```
[1, 0, 3, 0, 0, 5, 0, 0, 5, 0, 4, 0],
[0, 0, 5, 4, 0, 0, 4, 0, 0, 2, 1, 3],
[2, 4, 0, 1, 2, 0, 3, 0, 4, 3, 5, 0],
[0, 2, 4, 0, 5, 0, 0, 4, 0, 0, 2, 0],
[0, 0, 4, 3, 4, 2, 0, 0, 0, 0, 2, 5],
[1, 0, 3, 0, 3, 0, 0, 2, 0, 0, 4, 0]
```

When looking at the Input Matrix we can see that User 5 has the most similarity as User 3. And smaller similarities with User 4 and 7. Aswell as User 5 has the most difference with user 6 and 11. User 3 has rated the movie a 3 and User 4 and 7 has not rated the movie yet. User 6 and 11 has rated the movie a 5 and 4. With this is knowledge, aswell as the knowledge that User 4 and 7 has not rated the movie yet, then the Item-Item CF is the more acurate prediction, given with only my intuition and zero mathematical proff